In [1]:
!ls ../input/stratified-nwpu-resisc45-500-trainval7015/STRATIFIED_NWPU-RESISC45-500-TRAINVAL7015

train  val


In [2]:
!nvidia-smi

Mon Jun  2 06:42:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import tensorflow as tf

import keras
import keras_hub
import joblib
import os

2025-06-02 06:42:40.943662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748846561.161409      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748846561.219417      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
tf.config.list_logical_devices()

I0000 00:00:1748846575.587360      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

# Data Preparation

In [5]:
img_size = (224, 224)
batch_size = 10

In [6]:
path_ds = "../input/stratified-nwpu-resisc45-500-trainval7015/STRATIFIED_NWPU-RESISC45-500-TRAINVAL7015"

In [7]:
img_size = (224, 224)
batch_size = 10

# TRAIN DATA
print("================== TRAIN DATA")
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory = f"{path_ds}/train",
    shuffle = True,
    image_size = img_size,
    batch_size = batch_size
)
print(f"Len data within each batch: {len(train_data)}")


# VAL DATA
print("================== VAL DATA")
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory = f"{path_ds}/val",
    shuffle = True,
    image_size = img_size,
    batch_size = batch_size
)
print(f"Len data within each batch: {len(val_data)}")

================== TRAIN DATA
Found 15750 files belonging to 45 classes.
Len data within each batch: 1575
================== VAL DATA
Found 3375 files belonging to 45 classes.
Len data within each batch: 338


In [8]:
classes = train_data.class_names
classes

['airplane',
 'airport',
 'baseball_diamond',
 'basketball_court',
 'beach',
 'bridge',
 'chaparral',
 'church',
 'circular_farmland',
 'cloud',
 'commercial_area',
 'dense_residential',
 'desert',
 'forest',
 'freeway',
 'golf_course',
 'ground_track_field',
 'harbor',
 'industrial_area',
 'intersection',
 'island',
 'lake',
 'meadow',
 'medium_residential',
 'mobile_home_park',
 'mountain',
 'overpass',
 'palace',
 'parking_lot',
 'railway',
 'railway_station',
 'rectangular_farmland',
 'river',
 'roundabout',
 'runway',
 'sea_ice',
 'ship',
 'snowberg',
 'sparse_residential',
 'stadium',
 'storage_tank',
 'tennis_court',
 'terrace',
 'thermal_power_station',
 'wetland']

## Layers for Data Scaling and Augmentation

In [9]:
augmentation_layers = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomZoom(height_factor=0.1, width_factor=0.1),
    tf.keras.layers.RandomFlip("horizontal"),
])

rescaling_layer = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
])


train_data_no_augmentation = train_data.map(lambda x, y: (rescaling_layer(x), y))
val_data = val_data.map(lambda x, y: (rescaling_layer(x), y))
train_data_with_augmentation = train_data.map(lambda x, y: (augmentation_layers(x), y))


print(f"""Length of new data in each batch
train data: {len(train_data_no_augmentation)}
val data: {len(val_data)}
train data with augmentation: {len(train_data_with_augmentation)}
""")

Length of new data in each batch
train data: 1575
val data: 338
train data with augmentation: 1575



## Apply Data Prefetch

In [10]:
AUTOTUNE = tf.data.AUTOTUNE
train_data_no_augmentation = train_data_no_augmentation.prefetch(buffer_size=AUTOTUNE)
val_data = val_data.prefetch(buffer_size=AUTOTUNE)
train_data_with_augmentation = train_data_with_augmentation.prefetch(buffer_size=AUTOTUNE)

# Modelling

In [11]:
img_shape = (batch_size, *img_size, 3)
img_shape, img_shape[1:]

((10, 224, 224, 3), (224, 224, 3))

## Model Saving Callback (Epoch-based)

In [12]:
class EpochModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
    def __init__(self,
                 filepath,
                 frequency=1,
                 monitor="val_accuracy",
                 verbose=1,
                 save_best_only=False,
                 save_weights_only=False,
                 mode="max",
                 save_freq="epoch",
                 initial_value_threshold=None):

        super(EpochModelCheckpoint, self).__init__(filepath,
                                                   monitor,
                                                   verbose,
                                                   save_best_only,
                                                   save_weights_only,
                                                   mode,
                                                   save_freq,
                                                   initial_value_threshold)
        self.epochs_since_last_save = 0
        self.frequency = frequency

    def on_epoch_end(self, epoch, logs=None):
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save % self.frequency == 0:
            self._save_model(epoch=epoch, batch=None, logs=logs)

    def on_train_batch_end(self, batch, logs=None):
        pass

## Prepare Dir for Model's History

In [13]:
save_history_path = "../working/outputs/histories"
os.makedirs(save_history_path, exist_ok=True)

## ResNet-V2 Function Definition

In [14]:
def resnet50v2(epoch_size: int, augment: bool=False, frequency=5):
    model_name = f"{'base' if augment == False else 'aug'}_resnet50v2_{epoch_size}"

    model = tf.keras.applications.ResNet50V2(
        weights = "imagenet",
        include_top = False,
        input_shape = img_shape[1:]
    )

    model.trainable = True

    inputs = tf.keras.layers.Input(shape=img_shape[1:])
    x = model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    outputs = tf.keras.layers.Dense(len(classes))(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name=model_name)

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5, weight_decay=1e-2),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    callback = EpochModelCheckpoint(filepath=f"./outputs/{model_name}/{model_name}_epoch{{epoch:04d}}.keras", frequency=frequency)
    
    model.summary()
    print(f"\n##### Optimizer Func Information\n{model.optimizer.get_config()}")
    print(f"\n##### Loss Func Information\n{model.loss.get_config()}")

    return model, callback

In [ ]:
    model = tf.keras.applications.ConvNeXtTiny(
        include_top=False,
        include_preprocessing=False,
        weights="imagenet",
        input_shape=img_shape[1:],
        pooling=None,
        name="convnext_tiny",
    )

    model.trainable = True

## 15 Epochs No Augmentation

In [15]:
epoch_size = 15
base_resnet50v2_15, base_resnet50v2_15_ckpt_callback = resnet50v2(epoch_size=epoch_size, augment=False)

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "base_resnet50v2_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50v2 (Functional)              │ (None, 7, 7, 2048)          │      23,564,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 45)                  │          23,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,014,765 (91.61 MB)

 Trainable params: 23,969,325 (91.44 MB)

 Non-trainable params: 45,440 (177.50 KB)


##### Optimizer Func Information
{'name': 'adamw', 'learning_rate': 9.999999747378752e-06, 'weight_decay': 0.01, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}

##### Loss Func Information
{'name': 'sparse_categorical_crossentropy', 'reduction': 'sum_over_batch_size', 'from_logits': True, 'ignore_class': None}


In [16]:
%%time

history_base_resnet50v2_15 = base_resnet50v2_15.fit(
    train_data_no_augmentation,
    validation_data=val_data,
    epochs=epoch_size,
    callbacks=[base_resnet50v2_15_ckpt_callback]
)

Epoch 1/15


I0000 00:00:1748846646.415788      95 service.cc:148] XLA service 0x797058002280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748846646.416493      95 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1748846650.607797      95 cuda_dnn.cc:529] Loaded cuDNN version 90300


   2/1575 ━━━━━━━━━━━━━━━━━━━━ 1:55 73ms/step - accuracy: 0.1000 - loss: 3.7440   

I0000 00:00:1748846667.420667      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1575/1575 ━━━━━━━━━━━━━━━━━━━━ 184s 75ms/step - accuracy: 0.1314 - loss: 3.5716 - val_accuracy: 0.6803 - val_loss: 1.3760
Epoch 2/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 112s 71ms/step - accuracy: 0.6747 - loss: 1.3733 - val_accuracy: 0.8439 - val_loss: 0.5687
Epoch 3/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 112s 71ms/step - accuracy: 0.8243 - loss: 0.6524 - val_accuracy: 0.8714 - val_loss: 0.4450
Epoch 4/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 111s 71ms/step - accuracy: 0.8884 - loss: 0.4053 - val_accuracy: 0.8856 - val_loss: 0.3873
Epoch 5/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.9270 - loss: 0.2693
Epoch 5: saving model to ./outputs/base_resnet50v2_15/base_resnet50v2_15_epoch0005.keras
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 113s 71ms/step - accuracy: 0.9270 - loss: 0.2693 - val_accuracy: 0.8936 - val_loss: 0.3705
Epoch 6/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 111s 71ms/step - accuracy: 0.9526 - loss: 0.1727 - val_accuracy: 0.8913 - val_loss: 0.3704
Epoch 7/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 112s

In [17]:
joblib.dump(history_base_resnet50v2_15.history, os.path.join(save_history_path, "history_base_resnet50v2_15.joblib"))

['../working/outputs/histories/history_base_resnet50v2_15.joblib']

## 15 Epochs with Augmentation

In [18]:
epoch_size = 15
aug_resnet50v2_15, aug_resnet50v2_15_ckpt_callback = resnet50v2(epoch_size=epoch_size, augment=True)

Model: "aug_resnet50v2_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50v2 (Functional)              │ (None, 7, 7, 2048)          │      23,564,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 45)                  │          23,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,014,765 (91.61 MB)

 Trainable params: 23,969,325 (91.44 MB)

 Non-trainable params: 45,440 (177.50 KB)


##### Optimizer Func Information
{'name': 'adamw', 'learning_rate': 9.999999747378752e-06, 'weight_decay': 0.01, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}

##### Loss Func Information
{'name': 'sparse_categorical_crossentropy', 'reduction': 'sum_over_batch_size', 'from_logits': True, 'ignore_class': None}


In [19]:
%%time

history_aug_resnet50v2_15 = aug_resnet50v2_15.fit(
    train_data_with_augmentation,
    validation_data=val_data,
    epochs=epoch_size,
    callbacks=[aug_resnet50v2_15_ckpt_callback]
)

Epoch 1/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 182s 76ms/step - accuracy: 0.1429 - loss: 3.5147 - val_accuracy: 0.6904 - val_loss: 1.3355
Epoch 2/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 115s 73ms/step - accuracy: 0.6726 - loss: 1.3777 - val_accuracy: 0.8444 - val_loss: 0.5781
Epoch 3/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 114s 73ms/step - accuracy: 0.8026 - loss: 0.7194 - val_accuracy: 0.8785 - val_loss: 0.4284
Epoch 4/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 115s 73ms/step - accuracy: 0.8526 - loss: 0.5181 - val_accuracy: 0.8868 - val_loss: 0.3687
Epoch 5/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.8900 - loss: 0.3883
Epoch 5: saving model to ./outputs/aug_resnet50v2_15/aug_resnet50v2_15_epoch0005.keras
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 117s 74ms/step - accuracy: 0.8900 - loss: 0.3883 - val_accuracy: 0.9058 - val_loss: 0.3330
Epoch 6/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 115s 73ms/step - accuracy: 0.9112 - loss: 0.2981 - val_accuracy: 0.9084 - val_loss: 0.3235
Epoch 7/15
1575/1575 ━━━━━━━━━━━━━━━━

In [20]:
joblib.dump(history_aug_resnet50v2_15.history, os.path.join(save_history_path, "history_aug_resnet50v2_15.joblib"))

['../working/outputs/histories/history_aug_resnet50v2_15.joblib']